# Drug Related Deaths
## Individual Project
### Natasha Rivers
#### July 14, 2021

<hr style="border:1px solid black"> </hr>

## Project Scope

**Methodology Used**: Time Series

**Target Variable**: ???

**Primary Question**: Do drug related deaths change over time?

<hr style="border:3px solid black"> </hr>

## Imports

In [ ]:
#import libraries
import pandas as pd
import numpy as np

#visualization
import matplotlib.pyplot as plt
import seaborn as sns

#stats modules
import statsmodels.api as sm
from statsmodels.tsa.api import Holt
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import stats

#datetime utilities
from datetime import timedelta, datetime

#custom scripts
import acquire
import prepare

#remove warnings
import warnings
warnings.filterwarnings("ignore")

<hr style="border:2px solid black"> </hr>

## Acquire

In [ ]:
df = pd.read_csv('drug_deaths.csv')

In [ ]:
df.head()

In [ ]:
df.info()

<hr style="border:1px solid black"> </hr>

## Prepare

In [ ]:
#create columns for year, month, weekday
df['month'] = df.index.month_name()
df['day_of_week'] = df.index.day_name()
df['year'] = df.index.year

In [ ]:
#change dtype of 'dt'
#check current dtype- object
df.Date.dtypes

In [ ]:
#change data types
df.Date = df.Date.astype('datetime64[ns]')

In [ ]:
#set index and sort
df = df.set_index('Date').sort_index()

In [ ]:
#change ACCIDENT to Accident for uniformity
df = df.replace({'MannerofDeath': 'ACCIDENT'}, {'MannerofDeath': 'Accident'})
#change accident to Accident for uniformity
df = df.replace({'MannerofDeath': 'accident'}, {'MannerofDeath': 'Accident'})

In [ ]:
#change race options
df = df.replace({'Race': 'Asian, Other'}, {'Race': 'Other'})
df = df.replace({'Race': 'Asian Indian'}, {'Race': 'Other'})   
df = df.replace({'Race': 'Chinese'}, {'Race': 'Other'}) 
df = df.replace({'Race': 'Native American, Other'}, {'Race': 'Other'})
df = df.replace({'Race': 'Hawaiian'}, {'Race': 'Other'}) 
df = df.replace({'Race': 'Hispanic, White'}, {'Race': 'Hispanic'}) 
df = df.replace({'Race': 'Hispanic, Black'}, {'Race': 'Hispanic'}) 

In [ ]:
#drop multiple columns
df = df.drop(columns=['Unnamed: 0', 'DateType', 'ResidenceCounty','ResidenceState', 'DeathCounty', 'InjuryCounty','InjuryState', 'ResidenceCityGeo', 'InjuryCityGeo', 'Other', 'OtherSignifican', 'LocationifOther', 'DeathCityGeo', 'InjuryCity'])  